In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import Utility

### Public endpoint of Worldbank

In [ ]:
sparql = SPARQLWrapper("https://labs.tib.eu/sdm/worldbank_endpoint/sparql")

### Query to retrive values of a set of indicators by contry in a specific year

In [2]:
query = """

SELECT DISTINCT ?label_country ?label_indicator ?value ?year WHERE {

    ?country a <http://worldbank.org/Country> .
    ?country <http://www.w3.org/2002/07/owl#sameAs> ?sameAsCountry .
    ?country <http://worldbank.org/hasAnnualIndicatorEntry> ?annualIndicator .
    ?annualIndicator <http://www.w3.org/2002/07/owl#hasValue> ?value .
    ?annualIndicator <http://www.w3.org/2006/time#year> ?year .
    ?annualIndicator <http://worldbank.org/hasIndicator> ?indicator .
    ?indicator <http://www.w3.org/2000/01/rdf-schema#label> ?label_indicator .
    ?country <http://www.w3.org/2000/01/rdf-schema#label> ?label_country .
    
    FILTER(?indicator in (<http://worldbank.org/Indicator/NY.GDP.MKTP.CD>,
    <http://worldbank.org/Indicator/NE.DAB.TOTL.CD>, <http://worldbank.org/Indicator/NY.GDP.PCAP.CN>,
    <http://worldbank.org/Indicator/SI.POV.GINI>, <http://worldbank.org/Indicator/NV.IND.TOTL.ZS>,
    <http://worldbank.org/Indicator/EN.ATM.CO2E.PC>, <http://worldbank.org/Indicator/SH.STA.BASS.ZS>,
    <http://worldbank.org/Indicator/SH.STA.DIAB.ZS>, <http://worldbank.org/Indicator/SH.TBS.INCD>,
    <http://worldbank.org/Indicator/SP.DYN.LE00.IN>, <http://worldbank.org/Indicator/SP.POP.GROW>,
    <http://worldbank.org/Indicator/SH.HIV.INCD.TL.P3>, <http://worldbank.org/Indicator/SH.MED.BEDS.ZS>,
    <http://worldbank.org/Indicator/SH.PRG.ANEM>, <http://worldbank.org/Indicator/SH.PRV.SMOK>,
    <http://worldbank.org/Indicator/SH.STA.BASS.ZS>)
    AND (?year=2019))

}
"""

### Execute the query and store data into a pandas Dataframe

In [3]:
world_bank = []

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()


for r in results['results']['bindings']:
    row = {'country': r['label_country']['value'].replace(',', ''),
           'indicator': r['label_indicator']['value'].replace(',', ''),
           'value': r['value']['value']}
    
    world_bank.append(row)

world_bank = pd.DataFrame.from_dict(world_bank)
world_bank = world_bank.drop_duplicates()
display(world_bank.shape, world_bank.head(2))

(2328, 3)

,country,indicator,value
0,Pakistan,People using at least basic sanitation service...,66.5799
1,Pakistan,Incidence of tuberculosis (per 100000 people),263.0


In [4]:
world_bank.indicator.unique().tolist()

['People using at least basic sanitation services (% of population)',
 'Incidence of tuberculosis (per 100000 people)',
 'Life expectancy at birth total (years)',
 'Incidence of HIV all (per 1000 uninfected population)',
 'Population growth (annual %)',
 'Prevalence of anemia among pregnant women (%)',
 'Prevalence of current tobacco use (% of adults)',
 'CO2 emissions (metric tons per capita)',
 'Gross national expenditure (current US$)',
 'GDP (current US$)',
 'Industry (including construction) value added (% of GDP)',
 'GDP per capita (current LCU)',
 'Gini index',
 'Hospital beds (per 1000 people)']

In [5]:
len(world_bank.country.unique())

215

## Convert the indicator values into five categories

#### categories: 'low', 'mediumLow', 'medium', 'mediumHigh', 'high'.  (Higher is better)


In [6]:
country_ind = pd.DataFrame()
for indicator in world_bank.indicator.unique().tolist():
    lower = False
    if indicator in ['CO2 emissions (metric tons per capita)', 'Incidence of tuberculosis (per 100000 people)',
                    'Prevalence of anemia among pregnant women (%)',
                     'Incidence of HIV all (per 1000 uninfected population)',
                     'Prevalence of current tobacco use (% of adults)']:
        lower = True
    country_ind = pd.concat([country_ind, Utility.convert_to_category(world_bank, indicator, lower)])

country_ind

C:\Users\rivasa\GitHub\Coypu\FederateQuery\Utility.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["value"] = pd.to_numeric(df["value"], downcast="float")


,country,indicator,category_indicator
1754,Ethiopia,People using at least basic sanitation service...,low
5074,Chad,People using at least basic sanitation service...,low
3364,Madagascar,People using at least basic sanitation service...,low
3851,Niger,People using at least basic sanitation service...,low
920,Central African Republic,People using at least basic sanitation service...,low
...,...,...,...
4059,New Zealand,Hospital beds (per 1000 people),mediumLow
1518,Denmark,Hospital beds (per 1000 people),medium
2564,Iceland,Hospital beds (per 1000 people),medium
3200,Luxembourg,Hospital beds (per 1000 people),mediumHigh


In [7]:
country_ind = country_ind.drop_duplicates()
country_ind.shape

(2328, 3)

In [9]:
country_ind.to_csv('dataset/country_ind.csv', index=None)